In [1]:
#from Jon 

import pandas as pd
from bs4 import BeautifulSoup



In [2]:
# Read in the raw HTML file and load it into BeautifulSoup
with open('yourfile.html') as inf:
    soup = BeautifulSoup(inf.read(), 'html.parser')


In [3]:

# Dump out the HTML in an easier to read format
#print(soup.prettify())

In [4]:
# Find all results tables. Each table is wrapped by a div with class "brd-div"
# so we can find all those divs and then access the child table within each div
for div in soup.find_all('div', class_="brd-div"):
    table = div.find_next('table')
    # Now we have the table:
    # - the first row is the name of the trial
    # - the 2nd row is the "Last updated" row
    # - the 3rd row is the column headers
    # - all remaining rows are the data

   

In [ ]:
 # Get the first row
    trial_row = table.find_next('tr')
    # Find the a tag and retrieve the inner text - this is the trial name string
    trial = trial_row.find_next('a').contents[0]

   

In [ ]:
 # Get the next row (row 2)
    last_updated_row = trial_row.next_sibling
    # If you want the last updated time, pull it out here

    

In [ ]:
# Get the next row (row 3)
    col_header_row = last_updated_row.next_sibling
    # Iterate through the columns (td elements) in the row to extract the
    # column headers
    col_headers = []
    for td in col_header_row.find_all('td'):
        # The td elements we want have an inner div element with a class value
        # of gwt-HTML, the contents of that div are the label we want and the
        # label is split across two lines except the Rider/Horse column which
        # we need to treat as 2 separate columns
        if len(td.contents) == 3:
            # This is the Rider/Horse td
            col_headers.append(td.contents[0])
            col_headers.append(td.contents[2])
        else:
            inner_div = td.find('div', class_='gwt-HTML')
            if inner_div:
                label = inner_div.contents[0] + " " + inner_div.contents[2]
                if label == 'Rider Horse':
                    col_headers += label.split(' ')
                else:
                    col_headers.append(label)

  

In [ ]:
  # Now iterate over the remaining rows in the table which contain the data
    # Every row contains a div with class gwt-HTML like the header row except
    # the Rider/Horse column which has 2 values separated by a <br/> tag
    data_rows = []
    for row in col_header_row.next_siblings:
        data_row = []
        for td in row.find_all('td'):
            if len(td.contents) == 3:
                # This is the Rider/Horse td
                data_row.append(td.contents[0])
                # The Horse name is wrapped in <i> tags so we get the contents
                # of that inner tag
                data_row.append(td.contents[2].contents[0])
            else:
                inner_div = td.find('div', class_='gwt-HTML')
                if inner_div:
                    # The content of this div is the data value
                    data_row.append(inner_div.contents[0])
        data_rows.append(data_row)
    df = pd.DataFrame.from_records(data_rows, columns=col_headers)
    print(df)